In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

/home/ubuntu/sustaingym


In [2]:
from datetime import datetime, timedelta
import os
import pytz

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")

from sustaingym.data.load_moer import load_monthly_moer
from sustaingym.envs.evcharging import EVChargingEnv, RealTraceGenerator, GMMsTraceGenerator, DiscreteActionWrapper
from sustaingym.algorithms.evcharging.baselines import GreedyAlgorithm, MPC, RandomAlgorithm, RLAlgorithm, OfflineOptimal


test_ranges = (
    ('2019-05-01', '2019-08-31'),
    ('2019-09-01', '2019-12-31'),
    ('2020-02-01', '2020-05-31'),
    ('2021-05-01', '2021-08-31'),
)


In [ ]:
def plot_(site: SiteStr, period: DefaultPeriodStr) -> None:
    """Plot violin plots for baselines."""
    algs = ['greedy', 'random_continuous', 'random_discrete', 'offline_optimal']
    for window in [1, 3, 6, 12, 24]:
        algs.append(f'mpc_{window}')

    records = []
    for alg in algs:
        df = read_baseline(site, period, alg)
        reward = list(np.array(df.reward))
        for r in reward:
            records.append((alg, r))

    fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)
    df = pd.DataFrame.from_records(records, columns=['alg', 'reward'])
    sns.violinplot(data=df, x='alg', y='reward', ax=ax)
    ax.set(xlabel='Algorithm', ylabel="Reward ($)")
    ax.set_xticklabels(algs, rotation=30)
    fig.savefig(f'plots/violins_{site}_{period}.png', dpi=300, pad_inches=0)